In [ ]:
!pip install paho-mqtt

In [ ]:
import joblib
# Load the model from the file
model = joblib.load('/content/drive/MyDrive/Colab Notebooks/IoT/Final Project/random_forest_model/random_forest_model.pkl')

# Use the model to make predictions on new data
# Example: [temp1, temp2, hum1, press2]
X_new_data = np.array([27, 30, 80, 98300.24])
X_new_data = X_new_data.reshape(1, -1)

y_pred = model.predict(X_new_data)
y_prob = model.predict_proba(X_new_data)

print(y_pred)
print(y_prob)
print(X_new_data.shape)

[1]
[[0.2 0.8]]
(1, 4)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [10]:
import paho.mqtt.client as mqtt
import joblib
import numpy as np


# Define the topics
topics = [
    "room1/temperature1",
    "room1/humidity1",
    "room2/temperature2",
    "room2/pressure2"
]

# Dictionary to store the latest values from each topic
data = {
    "room1/temperature1": None,
    "room1/humidity1": None,
    "room2/temperature2": None,
    "room2/pressure2": None
}

# Define the callback for when a message is received
def on_message(client, userdata, message):
    global data
    topic = message.topic
    value = float(message.payload.decode())  # Convert message to float
    print(f"Received message: {value} on topic: {topic}")

    # Update the latest value for the topic
    if topic in data:
        data[topic] = value

    # Check if all data points are received
    if all(v is not None for v in data.values()):
        # Reorganize data into the expected input format [temp1, temp2, hum1, press2]
        input_vector = np.array([
            data["room1/temperature1"],  # temp1
            data["room2/temperature2"] - 0.9,  # temp2
            data["room1/humidity1"] - 4,     # hum1
            data["room2/pressure2"] - 350      # press2
        ]).reshape(1, -1)  # Reshape into column vector
        print(f"Input vector for model: {input_vector}")
        predict_anomaly(input_vector)  # Pass the data to the anomaly detection model

# Define the callback for when the client connects to the broker
def on_connect(client, userdata, flags, rc):
    print(f"Connected with result code {rc}")
    if rc == 0:
        # Subscribe to all topics
        for topic in topics:
            client.subscribe(topic, qos=1)
            print(f"Subscribed to topic: {topic}")
    else:
        print("Connection failed. Check your credentials or network settings.")

# Function to predict anomalies
def predict_anomaly(input_vector):
    # Predict using the Random Forest model (no scaling)
    prediction_prob = model.predict_proba(input_vector)  # Input data is already reshaped
    normal_score = prediction_prob[0,0]
    anomoly_score = prediction_prob[0,1]
    threshold = 0.5  # Adjust based on your model
    print("predictin_prob:",prediction_prob)

    # Check the prediction and determine if it's an anomaly or not
    if anomoly_score >= threshold:
        decision = "Anomaly detected!"  # If the anomaly score is higher than the threshold
    else:
        decision = "Normal data."

    # Print the decision locally
    print(decision)

    # Publish the decision to a new topic
    publish_topic = "anomaly/detection"
    client.publish(publish_topic, decision, qos=1)
    print(f"Published decision: '{decision}' to topic: '{publish_topic}'")

# Set up the MQTT client
broker = "cd6cdc287a1e4a5eb09c6350c2489f9c.s1.eu.hivemq.cloud"
port = 8883  # SSL port
username = "abutaherhivemq"
password = "Abutaherhivemq03"

client = mqtt.Client()

# Set credentials
client.username_pw_set(username, password)

# Enable SSL/TLS
client.tls_set()  # Enable TLS encryption

# Attach the callbacks
client.on_message = on_message
client.on_connect = on_connect

# Connect to the broker and subscribe to the topics
client.connect(broker, port, 60)

print(f"Subscribed to topics {topics} and listening for messages...")

# Start listening for messages
client.loop_forever()  # Blocking loop to listen indefinitely


<ipython-input-10-dd8d1889e9ca>:85: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Subscribed to topics ['room1/temperature1', 'room1/humidity1', 'room2/temperature2', 'room2/pressure2'] and listening for messages...
Connected with result code 0
Subscribed to topic: room1/temperature1
Subscribed to topic: room1/humidity1
Subscribed to topic: room2/temperature2
Subscribed to topic: room2/pressure2
Received message: 22.43 on topic: room2/temperature2
Received message: 21.9 on topic: room1/temperature1
Received message: 99638.59 on topic: room2/pressure2
Received message: 11.8 on topic: room1/humidity1
Input vector for model: [[2.190000e+01 2.153000e+01 7.800000e+00 9.928859e+04]]
predictin_prob: [[0.69 0.31]]
Normal data.
Published decision: 'Normal data.' to topic: 'anomaly/detection'


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Received message: 22.43 on topic: room2/temperature2
Input vector for model: [[2.190000e+01 2.153000e+01 7.800000e+00 9.928859e+04]]
predictin_prob: [[0.69 0.31]]
Normal data.
Published decision: 'Normal data.' to topic: 'anomaly/detection'
Received message: 21.9 on topic: room1/temperature1
Input vector for model: [[2.190000e+01 2.153000e+01 7.800000e+00 9.928859e+04]]
predictin_prob: [[0.69 0.31]]
Normal data.
Published decision: 'Normal data.' to topic: 'anomaly/detection'
Received message: 99638.59 on topic: room2/pressure2
Input vector for model: [[2.190000e+01 2.153000e+01 7.800000e+00 9.928859e+04]]
predictin_prob: [[0.69 0.31]]
Normal data.
Published decision: 'Normal data.' to topic: 'anomaly/detection'
Received message: 11.8 on topic: room1/humidity1
Input vector for model: [[2.190000e+01 2.153000e+01 7.800000e+00 9.928859e+04]]
predictin_prob: [[0.69 0.31]]
Normal data.
Published decision: 'Normal data.' to topic: 'anomaly/detection'


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


...

Received message: 24.72 on topic: room2/temperature2
Input vector for model: [[2.350000e+01 2.382000e+01 2.260000e+01 9.928927e+04]]
predictin_prob: [[0.31 0.69]]
Anomaly detected!
Published decision: 'Anomaly detected!' to topic: 'anomaly/detection'
Received message: 23.4 on topic: room1/temperature1
Input vector for model: [[2.340000e+01 2.382000e+01 2.260000e+01 9.928927e+04]]
predictin_prob: [[0.31 0.69]]
Anomaly detected!
Published decision: 'Anomaly detected!' to topic: 'anomaly/detection'
Received message: 23.4 on topic: room1/temperature1
Input vector for model: [[2.340000e+01 2.382000e+01 2.260000e+01 9.928927e+04]]
predictin_prob: [[0.31 0.69]]
Anomaly detected!
Published decision: 'Anomaly detected!' to topic: 'anomaly/detection'
Received message: 99637.66 on topic: room2/pressure2
Input vector for model: [[2.340000e+01 2.382000e+01 2.260000e+01 9.928766e+04]]
predictin_prob: [[0.31 0.69]]
Anomaly detected!
Published decision: 'Anomaly detected!' to topic: 'anomaly/detection

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Received message: 24.67 on topic: room2/temperature2
Input vector for model: [[2.340000e+01 2.377000e+01 2.200000e+01 9.928766e+04]]
predictin_prob: [[0.31 0.69]]
Anomaly detected!
Published decision: 'Anomaly detected!' to topic: 'anomaly/detection'
Received message: 24.62 on topic: room2/temperature2
Input vector for model: [[2.340000e+01 2.372000e+01 2.200000e+01 9.928766e+04]]
predictin_prob: [[0.31 0.69]]
Anomaly detected!
Published decision: 'Anomaly detected!' to topic: 'anomaly/detection'
Received message: 24.58 on topic: room2/temperature2
Input vector for model: [[2.340000e+01 2.368000e+01 2.200000e+01 9.928766e+04]]
predictin_prob: [[0.31 0.69]]
Anomaly detected!
Published decision: 'Anomaly detected!' to topic: 'anomaly/detection'
Received message: 23.4 on topic: room1/temperature1
Input vector for model: [[2.340000e+01 2.368000e+01 2.200000e+01 9.928766e+04]]
predictin_prob: [[0.31 0.69]]
Anomaly detected!
Published decision: 'Anomaly detected!' to topic: 'anomaly/detectio

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X

KeyboardInterrupt: 